# Bulk Assign Glossary Terms to Assets with PVW CLI

This notebook demonstrates how to automate the assignment of glossary terms to multiple assets in Microsoft Purview using the PVW CLI. It reads a CSV mapping of entities to terms and performs bulk assignment operations.

**For detailed CLI and API documentation, see the main [README](../..//README.md).**


# Bulk Assign Glossary Terms to Assets

**Description**
- This notebook will read a CSV file which has a list of entities mapped to a glossary term.
- The notebook will then loop through each term and bulk assign the entities.

**Prerequisites**
- You need to have a CSV file (e.g. entities_to_terms.csv) with three columns (guid, qualifiedName, termFormalName).
- The first two columns (guid, qualifiedName) is a list of entities, the third column is the glossary term you would like to assign (termFormalName).
- See the entities_to_term.csv sample file (purviewcli\samples\csv).

In [ ]:
# ENVIRONMENT VARIABLES

# 1. Update PURVIEW_ACCOUNT_NAME with your Purview account name
%env PURVIEW_ACCOUNT_NAME=YOUR_PURVIEW_ACCOUNT_NAME

# 2. Uncomment and set the variables below if you plan to use a Service Principal
# %env AZURE_CLIENT_ID=YOUR_CLIENT_ID
# %env AZURE_TENANT_ID=YOUR_TENANT_ID
# %env AZURE_CLIENT_SECRET=YOUR_CLIENT_SECRET

# 3. Update the path to your local entities_to_term.csv file path
PATH_TO_CSV = "../csv/entities_to_term.csv"

In [ ]:
# Helper Methods
import json
def getJSON(raw_output):
    output = ''.join(raw_output)
    json_obj = json.loads(output)
    return json_obj

In [ ]:
# 1. Convert mapping from CSV to Dictionary
import csv
terms = {}
with open(PATH_TO_CSV) as fp:
    reader = csv.reader(fp, delimiter=",", quotechar='"')
    next(reader, None) # skip headers
    for row in reader:
        guid, qualifiedName, termFormalName = row[0],row[1],row[2]
        if termFormalName in terms:
            terms[termFormalName].append(guid)
        else:
            terms[termFormalName] = []
            terms[termFormalName].append(guid)
print(terms)

In [ ]:
# 2. Get Glossary
glossary = !pvw glossary read
glossary = getJSON(glossary)

In [ ]:
# 3. Get Term GUID function
def getTermGuid(termFormalName, glossary):
    termGuid = None
    for term in glossary[0]['terms']:
        if term['displayText'] == termFormalName:
            termGuid = term['termGuid']
    return termGuid

In [ ]:
# 4. Assign terms to entities
for term in terms:
    termGuid = getTermGuid(term, glossary)
    numberOfEntities = len(terms[term])
    if numberOfEntities > 0:
        payload = []
        for entity in terms[term]:
            item = {"guid": entity}
            payload.append(item)
        with open("payload.json", "w") as outfile:
            json.dump(payload, outfile, indent=4, sort_keys=False)
        !pvw glossary createTermsAssignedEntities --termGuid {termGuid} --payloadFile "payload.json"